In [1]:
import pandas as pd
import urllib.request, json
import plotly_express as px
import plotly.figure_factory as ff
from pyxirr import xirr

In [2]:
import yfinance as yf

In [ ]:
with urllib.request.urlopen('https://api.mfapi.in/mf') as url:
    data = json.load(url)
df_mfs = pd.DataFrame(data)

In [ ]:
df_mfs[df_mfs.schemeName.str.contains('HDFC Flex')]

In [ ]:
df_mfs[df_mfs.schemeName.str.startswith('Parag')]

In [ ]:
df_sel_name = df_mfs.schemeName[0]

In [ ]:
df_sel_code = df_mfs[df_mfs['schemeName'] == df_sel_name]['schemeCode'][0]
df_sel_code

In [ ]:
def get_nav(scheme_code = '122639'):
    # scheme_code = '122639'
    mf_url = 'https://api.mfapi.in/mf/' + scheme_code
    with urllib.request.urlopen(mf_url) as url:
        data = json.load(url)

    df_navs = pd.DataFrame(data['data'])
    df_navs['date'] = pd.to_datetime(df_navs.date, format='%d-%m-%Y')
    df_navs['nav'] = df_navs['nav'].astype(float)
    df_navs = df_navs.sort_values(['date']).set_index(['date'])
    df_dates = pd.DataFrame(pd.date_range(start=df_navs.index.min(), end=df_navs.index.max()), columns=['date']).set_index(['date'])
    df_navs = df_navs.join(df_dates, how='outer').ffill().reset_index()
    return df_navs

In [ ]:
df_navs = get_nav()

In [ ]:
base_date = '2015-01-01'
base_nav = df_navs.set_index(['date']).loc[base_date:].nav[0]
df_sel_navs = df_navs[df_navs['date'] >= base_date].copy()
df_sel_navs

In [ ]:
px.line(df_navs, x = 'date', y='nav', log_y=True)

In [ ]:
def get_cagr(df_navs_orig, num_years = 1):
    df_navs = df_navs_orig.copy()
    df_navs['prev_nav'] = df_navs.nav.shift(365 * num_years)
    df_navs = df_navs.dropna()
    df_navs['returns'] = df_navs['nav'] / df_navs['prev_nav'] - 1
    df_navs['cagr'] = 100 * ((1 + df_navs['returns']) ** (1 / num_years) - 1)
    df_navs['years'] = num_years
    df_cagr = df_navs[['date', 'years', 'cagr']]
    return df_cagr

In [ ]:
all_names = ['Parag Parikh Flexi Cap Fund - Direct Plan - Growth', 'HDFC Equity Fund - Growth Option']

In [ ]:
list_navs = []
for name in all_names:
    code = df_mfs[df_mfs['schemeName'] == name].schemeCode.to_list()[0]
    df_nav_comp = get_nav(str(code))
    df_nav_comp = df_nav_comp.set_index('date')
    df_nav_comp = df_nav_comp.rename(columns={'nav': name})
    list_navs.append(df_nav_comp)

df_nav_all = pd.concat(list_navs, axis=1).dropna()
df_rebased = df_nav_all.div(df_nav_all.iloc[0]).reset_index()
df_rebased_long = pd.melt(df_rebased, id_vars='date', value_vars=all_names, var_name='mf', value_name='nav')

In [ ]:
df_nav = df_nav_comp.reset_index()
df_nav.columns = ['date', 'nav']

In [ ]:
df_nav.dtypes

In [ ]:
df_nav.head()

In [ ]:
start_date = pd.to_datetime('2006-05-01')
end_date = pd.to_datetime('2020-04-01')

In [ ]:
df_nav[df_nav['date'] >= start_date].head()

In [ ]:
df_dates = pd.DataFrame(pd.date_range(start=start_date, end=end_date, freq='M'))
df_dates.columns = ['date']

In [ ]:
df_cf = df_nav.merge(df_dates, on='date')
df_cf['amount'] = 1000
df_cf['units'] = df_cf['amount'] / df_cf['nav']

In [ ]:
df_cf['cum_units'] = df_cf['units'].cumsum()
df_cf['inv_value'] = df_cf['cum_units'] * df_cf['nav']
df_cf['cum_amount'] = df_cf['amount'].cumsum()

In [ ]:
df_daily_dates = pd.DataFrame(pd.date_range(start=df_cf['date'].min(), end=df_cf['date'].max(), freq='D'))
df_daily_dates.columns = ['date']
df_cfs = df_cf.merge(df_daily_dates, on='date', how='right').sort_values(['date'])
df_cfs = df_cfs.ffill()
df_cf_long = pd.melt(df_cfs[['date', 'cum_amount', 'inv_value']], id_vars=['date'], value_vars=['cum_amount', 'inv_value'])

In [ ]:
px.line(df_cf_long, x='date', y='value', color='variable')

In [ ]:
df_investment = df_cf[['date', 'amount']]

In [ ]:
df_redemption = pd.DataFrame([{'date': df_cf.iloc[-1:].date.values[0], 'amount': -df_cf['units'].sum() * df_cf.iloc[-1:].nav.values[0]}])

In [ ]:
df_irr = pd.concat([df_investment, df_redemption]).reset_index(drop=True)
df_irr

In [ ]:
xirr(df_irr[['date', 'amount']]) * 100

In [ ]:
float('293.323')

In [ ]:
years = [x + 1 for x in range(9)]
list_cagr = []
for y in years:
    df_cagr = get_cagr(df_navs, y)
    list_cagr.append(df_cagr)
df_cagrs = pd.concat(list_cagr)

In [ ]:
df_cagrs.dtypes

In [ ]:
import datetime
import numpy as np

In [ ]:
np.datetime64(datetime.datetime(2002, 6, 28, 1, 0))

In [ ]:
px.line(df_cagrs, x='date', y='cagr', color='years')

In [ ]:
df_cagrs.head()

In [ ]:
dfx = df_cagrs[['date', 'years', 'cagr']].groupby('years').describe().reset_index()
dfx.columns = [[a for (a, b) in dfx.columns][0]] + [a for a in dfx.columns.droplevel()][1:]
dfx

In [ ]:
df_cagr

In [ ]:
all_names

In [ ]:
list_navs = []
list_cagrs = []
for name in all_names:
    code = df_mfs[df_mfs['schemeName'] == name].schemeCode.to_list()[0]
    df_nav_comp = get_nav(str(code))
    years = [x for x in range(1, 11)]
    list_cagr = []
    for y in years:
        df_cagr = get_cagr(df_nav_comp, y)
        list_cagr.append(df_cagr)
    df_cagrs_comp = pd.concat(list_cagr)

    df_nav_comp = df_nav_comp.set_index('date')
    df_nav_comp = df_nav_comp.rename(columns={'nav': name})
    list_navs.append(df_nav_comp)

    df_cagrs_comp = df_cagrs_comp.set_index(['date', 'years'])
    df_cagrs_comp = df_cagrs_comp.rename(columns={'cagr': name})
    list_cagrs.append(df_cagrs_comp)

df_nav_all = pd.concat(list_navs, axis=1).dropna()
df_cagr_all = pd.concat(list_cagrs, axis=1).dropna()

In [ ]:
df_nav_all

In [ ]:
df_cagr_all

In [ ]:
df_navs_date = df_nav_all.reset_index()
min_date = df_navs_date['date'].min()
max_date = df_navs_date['date'].max()
from_date = "2020-03-23"
df_nav_all = df_navs_date[df_navs_date['date'] >= np.datetime64(from_date)].set_index('date')

In [ ]:
df_rebased = df_nav_all.div(df_nav_all.iloc[0]).reset_index()
df_rebased_long = pd.melt(df_rebased, id_vars='date', value_vars=all_names, var_name='mf', value_name='nav')

In [ ]:
df_rebased

In [ ]:
df_cagr_wide = df_cagr_all.reset_index()
df_cagr_long = pd.melt(df_cagr_wide, id_vars=['date', 'years'], value_vars=all_names, var_name='mf', value_name='cagr')

In [ ]:
df_cagr_long

In [ ]:
px.density_contour(df_cagrs.set_index(['date']), x='cagr', color='years')

In [ ]:
# df_cagrs[df_cagrs['years'] == 9]
px.histogram(df_cagrs, x='cagr', color='years')

In [ ]:
df_cagrs['cagr_bucket'] = round(df_cagrs['cagr'] / 1, 0) * 1

In [ ]:
df_total = pd.DataFrame(df_cagrs.groupby(['years']).cagr.count()).reset_index()
df_total.columns = ['years', 'total']

In [ ]:
df_counts = pd.DataFrame(df_cagrs.groupby(['years', 'cagr_bucket']).cagr.count()).reset_index()
df_counts.columns = ['years', 'cagr_bucket', 'count']

In [ ]:
df_x = df_counts.merge(df_total, on='years', how='left')
df_x['pct'] = df_x['count'] / df_x['total']

In [ ]:
df_x['years'] = df_x['years'].astype(str)
px.bar(df_x, x='cagr_bucket', y='pct', color='years', barmode='overlay')

In [ ]:
df_x

In [ ]:
with open('../data/mf_codes.txt', 'r') as fp:
    list_code = []
    line = fp.readline()
    while line:
        words = line.strip().split(';')
        if len(words) > 5:
            list_code.append([words[i] for i in [0, 1, 3]])
        line = fp.readline()

In [ ]:
df_codes = pd.DataFrame(list_code)

In [ ]:
# df_codes.columns = ['code', 'isin', 'none', 'name', 'nav', 'date']
df_codes.columns = ['schemeCode', 'schemeISIN', 'schemeName']

In [ ]:
with urllib.request.urlopen('https://api.mfapi.in/mf') as url:
    data = json.load(url)
df_mfs = pd.DataFrame(data)

In [ ]:
# SWP in progress

In [83]:
list_scr = pd.read_html('https://www.screener.in/company/LGBBROSLTD/consolidated/')

In [84]:
len(list_scr)

10

In [85]:
list_scr[0]

,Unnamed: 0,Dec 2019,Mar 2020,Jun 2020,Sep 2020,Dec 2020,Mar 2021,Jun 2021,Sep 2021,Dec 2021,Mar 2022,Jun 2022,Sep 2022,Dec 2022
0,Sales +,407,356,195,417,498,499,393,577,574,559,521,578,581
1,Expenses +,351,322,180,359,407,410,331,470,458,454,436,476,474
2,Operating Profit,55,34,14,58,91,89,61,107,116,105,85,102,107
3,OPM %,14%,10%,7%,14%,18%,18%,16%,18%,20%,19%,16%,18%,18%
4,Other Income +,6,8,1,6,2,12,4,20,3,8,13,6,7
5,Interest,4,3,3,3,2,2,2,2,3,3,1,1,2
6,Depreciation,20,21,20,21,21,21,20,21,21,21,20,20,20
7,Profit before tax,37,18,-8,41,70,78,43,104,95,90,77,87,93
8,Tax %,25%,30%,25%,33%,23%,25%,23%,25%,25%,29%,24%,23%,22%
9,Net Profit,28,13,-6,27,54,58,33,78,71,64,58,67,72


In [144]:
df_qly_sales = list_scr[0].rename({'Unnamed: 0': 'Metric'}, axis=1)
df_yly_sales = list_scr[1].rename({'Unnamed: 0': 'Metric'}, axis=1)

In [145]:
df_yly_sales

,Metric,Mar 2011,Mar 2012,Mar 2013,Mar 2014,Mar 2015,Mar 2016,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,TTM
0,Sales +,704,898,939,1085,1151,1175,1258,1418,1688,1543,1609,2102,2239
1,Expenses +,616,794,850,959,1009,1037,1095,1222,1479,1355,1355,1713,1840
2,Operating Profit,87,104,89,126,142,138,162,196,209,187,254,389,399
3,OPM %,12%,12%,10%,12%,12%,12%,13%,14%,12%,12%,16%,19%,18%
4,Other Income +,2,2,4,5,9,4,10,5,15,22,21,35,35
5,Interest,15,18,23,18,18,18,16,12,13,16,11,9,7
6,Depreciation,26,29,30,33,39,46,53,58,68,79,83,83,80
7,Profit before tax,48,59,40,80,94,78,103,131,143,115,180,332,347
8,Tax %,4%,25%,19%,19%,22%,17%,28%,33%,32%,22%,26%,26%,NaN
9,Net Profit,46,44,33,66,74,66,75,89,100,91,133,246,261


In [146]:
df_qly_sales.Metric[3]

'OPM %'

In [124]:
df_qly = df_qly_sales[df_qly_sales['Metric'] == df_qly_sales.Metric[0]].transpose()
df_qly = df_qly.reset_index()
df_qly.columns = ['Date', 'Sales (Cr)']
df_qly = df_qly[df_qly['Date'] != 'Metric']
df_qly['Sales (Cr)'] = df_qly['Sales (Cr)'].astype(float)

In [134]:
px.bar(df_qly, x='Date', y='Sales (Cr)', title='Quarterly Sales')

In [136]:
df_qly['QoQ Growth (%)'] = df_qly['Sales (Cr)'].pct_change() * 100
df_qly['YoY Growth (%)'] = df_qly['Sales (Cr)'].pct_change(4) * 100

In [142]:
px.bar(df_qly, x='Date', y='QoQ Growth (%)', title='Quarterly Sales (QoQ)')

In [143]:
px.bar(df_qly.dropna(), x='Date', y='YoY Growth (%)', title='Quarterly Sales (YoY)')

In [3]:
help(yf.download)

Help on function download in module yfinance.multi:

download(tickers, start=None, end=None, actions=False, threads=True, group_by='column', auto_adjust=False, back_adjust=False, progress=True, period='max', show_errors=True, interval='1d', prepost=False, proxy=None, rounding=False, timeout=None, **kwargs)
    Download yahoo tickers
    :Parameters:
        tickers : str, list
            List of tickers to download
        period : str
            Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            Either Use period parameter or use start and end
        interval : str
            Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            Intraday data cannot extend last 60 days
        start: str
            Download start date string (YYYY-MM-DD) or _datetime.
            Default is 1900-01-01
        end: str
            Download end date string (YYYY-MM-DD) or _datetime.
            Default is now
        group_by : str
            Group by 'ticker' o

In [4]:
infy = yf.Ticker('INFY.NS')

In [4]:
msft = yf.Ticker('MSFT')

In [11]:
infy.actions

,Dividends,Stock Splits
Date,,
1996-04-30,0.006836,0.0
1996-10-28,0.002930,0.0
1997-05-23,0.007813,0.0
1997-08-19,0.000000,2.0
1997-12-17,0.005859,0.0
...,...,...
2020-10-23,12.000000,0.0
2021-05-31,15.000000,0.0
2021-10-26,15.000000,0.0


In [17]:
infy.splits

Date
1997-08-19    2.0
1999-02-10    2.0
2000-01-24    2.0
2000-01-27    2.0
2004-07-01    4.0
2006-07-13    2.0
2014-12-02    2.0
2015-06-15    2.0
2018-09-04    2.0
Name: Stock Splits, dtype: float64

In [18]:
infy.dividends

Date
1996-04-30     0.006836
1996-10-28     0.002930
1997-05-23     0.007813
1997-12-17     0.005859
1998-04-30     0.017578
1998-11-12     0.009766
1999-05-20     0.039063
1999-10-28     0.023438
2000-04-28     0.046875
2000-10-25     0.039063
2001-04-27     0.117188
2001-10-18     0.117188
2002-05-21     0.195313
2002-10-30     0.195313
2003-05-28     0.226563
2003-10-16     0.226563
2004-05-26     1.562500
2004-10-18     0.312500
2005-06-01     0.406250
2005-10-17     0.406250
2006-05-25     2.406250
2006-10-19     0.625000
2007-06-06     0.812500
2007-10-18     0.750000
2008-05-29     3.406250
2008-10-16     1.250000
2009-06-04     1.687500
2009-10-15     1.250000
2010-05-26     1.875000
2010-10-21     3.750000
2011-05-26     2.500000
2011-10-20     1.875000
2012-05-24     1.250000
2012-10-18     1.875000
2013-05-30     3.375000
2013-10-17     2.500000
2014-05-29     5.375000
2014-10-16     3.750000
2015-06-15     7.375000
2015-10-16     5.000000
2016-06-09     7.125000
2016-10-21 

In [27]:
import yahoo_fin.stock_info as si

In [32]:
bl = si.get_balance_sheet('nflx')

In [36]:
msft.info.keys()

dict_keys(['regularMarketPrice', 'preMarketPrice', 'logo_url'])

In [3]:
yf.__version__

'0.2.12'

In [9]:
from yahooquery import Ticker

In [78]:
ticker = Ticker('matrimony.ns')

In [73]:
income_statement = ticker.income_statement('q') 

In [74]:
income_statement

,asOfDate,periodType,currencyCode,BasicAverageShares,BasicEPS,CostOfRevenue,DilutedAverageShares,DilutedEPS,DilutedNIAvailtoComStockholders,EBIT,...,SpecialIncomeCharges,TaxEffectOfUnusualItems,TaxProvision,TaxRateForCalcs,TotalExpenses,TotalOperatingIncomeAsReported,TotalRevenue,TotalUnusualItems,TotalUnusualItemsExcludingGoodwill,WriteOff
symbol,,,,,,,,,,,,,,,,,,,,,
infy.ns,2021-12-31,3M,USD,4.190866e+09,0.18,2.856000e+09,4.198924e+09,0.18,7.740000e+08,1.066000e+09,...,-7000000.0,1.603399e+06,2.830000e+08,0.267233,3.245000e+09,9.980000e+08,4.250000e+09,6000000.0,6000000.0,7000000.0
infy.ns,2022-03-31,3M,USD,4.191743e+09,0.18,2.955000e+09,4.199791e+09,0.18,7.520000e+08,1.005000e+09,...,-4000000.0,5.155311e+06,2.450000e+08,0.245491,3.356000e+09,9.200000e+08,4.280000e+09,21000000.0,21000000.0,4000000.0
infy.ns,2022-06-30,3M,USD,4.193748e+09,0.16,3.144000e+09,4.199492e+09,0.16,6.890000e+08,9.750000e+08,...,-6000000.0,3.170455e+06,2.790000e+08,0.288223,3.550000e+09,8.880000e+08,4.444000e+09,11000000.0,11000000.0,6000000.0
infy.ns,2022-09-30,3M,USD,4.155556e+09,0.18,3.170000e+09,4.155556e+09,0.18,7.480000e+08,9.790000e+08,...,NaN,0.000000e+00,2.950000e+08,0.282567,3.576000e+09,9.790000e+08,4.555000e+09,NaN,NaN,NaN
infy.ns,2022-12-31,3M,USD,4.210526e+09,0.19,3.230000e+09,4.210526e+09,0.19,8.000000e+08,1.001000e+09,...,NaN,0.000000e+00,2.850000e+08,0.262673,3.658000e+09,1.001000e+09,4.659000e+09,NaN,NaN,NaN
infy.ns,2022-12-31,TTM,USD,NaN,NaN,1.249900e+10,NaN,NaN,2.989000e+09,4.096000e+09,...,NaN,9.975092e+06,1.104000e+09,0.269597,1.412600e+10,3.788000e+09,1.793700e+10,37000000.0,37000000.0,NaN


In [23]:
ticker.financial_data

{'ltim.ns': {'maxAge': 86400,
  'currentPrice': 4726.0,
  'recommendationKey': 'none',
  'totalCash': 73333997568,
  'totalCashPerShare': 247.75,
  'ebitda': 36321001472,
  'totalDebt': 15285999616,
  'quickRatio': 2.64,
  'currentRatio': 3.033,
  'totalRevenue': 211805995008,
  'debtToEquity': 9.656,
  'revenuePerShare': 1207.679,
  'returnOnAssets': 0.11995,
  'returnOnEquity': 0.22900999,
  'grossProfits': 33389000000,
  'freeCashflow': -2523249920,
  'operatingCashflow': 18680999936,
  'earningsGrowth': -0.031,
  'revenueGrowth': 1.083,
  'grossMargins': 0.22499001,
  'ebitdaMargins': 0.17148,
  'operatingMargins': 0.15604,
  'profitMargins': 0.12991999,
  'financialCurrency': 'INR'}}

In [24]:
ticker.earnings_trend

{'ltim.ns': 'No fundamentals data found for any of the summaryTypes=earningsTrend'}

In [25]:
ticker.earnings

{'ltim.ns': 'No fundamentals data found for any of the summaryTypes=earnings'}

In [28]:
ticker.income_statement()

,asOfDate,periodType,currencyCode,Amortization,BasicAverageShares,BasicEPS,CostOfRevenue,DepreciationAndAmortizationInIncomeStatement,DepreciationIncomeStatement,DilutedAverageShares,...,SpecialIncomeCharges,TaxEffectOfUnusualItems,TaxProvision,TaxRateForCalcs,TotalExpenses,TotalOtherFinanceCost,TotalRevenue,TotalUnusualItems,TotalUnusualItemsExcludingGoodwill,WriteOff
symbol,,,,,,,,,,,,,,,,,,,,,
ltim.ns,2019-03-31,12M,INR,NaN,172912774.0,87.67,5.529200e+10,1.472000e+09,1.472000e+09,175381719.0,...,-23000000.0,-5.819000e+06,5.123000e+09,0.253000,7.707400e+10,1000000.0,9.445800e+10,-2.300000e+07,-2.300000e+07,23000000.0
ltim.ns,2020-03-31,12M,INR,750000000.0,173823653.0,87.45,6.576600e+10,2.730000e+09,1.980000e+09,175508270.0,...,-89000000.0,3.557366e+08,4.824000e+09,0.240851,9.113400e+10,19000000.0,1.087860e+11,1.477000e+09,1.477000e+09,89000000.0
ltim.ns,2021-03-31,12M,INR,778000000.0,174460220.0,110.98,7.594300e+10,3.325000e+09,2.547000e+09,175596111.0,...,440000000.0,2.679662e+08,6.500000e+09,0.251140,9.964100e+10,10000000.0,1.236980e+11,1.067000e+09,1.067000e+09,131000000.0
ltim.ns,2022-03-31,12M,INR,950000000.0,175073125.0,131.19,9.918100e+10,3.549000e+09,2.599000e+09,175073125.0,...,-78000000.0,5.300379e+08,7.989000e+09,0.257926,1.295740e+11,55000000.0,1.566870e+11,2.055000e+09,2.055000e+09,78000000.0
ltim.ns,2022-12-31,TTM,INR,NaN,NaN,NaN,1.357460e+11,4.598000e+09,3.648000e+09,NaN,...,NaN,4.659798e+08,8.777000e+09,0.241691,1.781780e+11,NaN,2.118060e+11,1.928000e+09,1.928000e+09,NaN


In [65]:
ticker.all_modules['infy.ns'].keys()

dict_keys(['assetProfile', 'recommendationTrend', 'cashflowStatementHistory', 'indexTrend', 'defaultKeyStatistics', 'industryTrend', 'quoteType', 'incomeStatementHistory', 'fundOwnership', 'summaryDetail', 'insiderHolders', 'calendarEvents', 'price', 'balanceSheetHistory', 'earningsTrend', 'institutionOwnership', 'majorHoldersBreakdown', 'balanceSheetHistoryQuarterly', 'earningsHistory', 'esgScores', 'summaryProfile', 'netSharePurchaseActivity', 'insiderTransactions', 'sectorTrend', 'incomeStatementHistoryQuarterly', 'cashflowStatementHistoryQuarterly', 'earnings', 'financialData'])

In [70]:
ticker.all_modules['infy.ns']['recommendationTrend']

{'trend': [{'period': '0m',
   'strongBuy': 0,
   'buy': 0,
   'hold': 0,
   'sell': 0,
   'strongSell': 0},
  {'period': '-1m',
   'strongBuy': 9,
   'buy': 12,
   'hold': 18,
   'sell': 8,
   'strongSell': 1},
  {'period': '-2m',
   'strongBuy': 8,
   'buy': 13,
   'hold': 16,
   'sell': 10,
   'strongSell': 1},
  {'period': '-3m',
   'strongBuy': 17,
   'buy': 14,
   'hold': 14,
   'sell': 5,
   'strongSell': 0}],
 'maxAge': 86400}

In [66]:
pd.DataFrame(ticker.all_modules['infy.ns']['balanceSheetHistory']['balanceSheetStatements'])

,maxAge,endDate,cash,shortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,longTermInvestments,propertyPlantEquipment,...,minorityInterest,totalCurrentLiabilities,totalLiab,commonStock,retainedEarnings,treasuryStock,capitalSurplus,otherStockholderEquity,totalStockholderEquity,netTangibleAssets
0,1,2022-03-31,2305000000,880000000,4609000000,25000000,783000000,8865000000,1801000000,2429000000,...,53000000,4433000000,5561000000,328000000,12842000000,-3566000000,337000000,-3566000000,9941000000,8899000000
1,1,2021-03-31,3380000000,320000000,3776000000,19000000,653000000,8307000000,1623000000,2519000000,...,60000000,3264000000,4323000000,332000000,12995000000,-3244000000,359000000,-3244000000,10442000000,9327000000
2,1,2020-03-31,2465000000,615000000,3479000000,19000000,506000000,7212000000,547000000,2414000000,...,55000000,2757000000,3559000000,332000000,11608000000,-3599000000,305000000,-3599000000,8646000000,7696000000
3,1,2019-03-31,2829000000,958000000,3148000000,16000000,585000000,7644000000,670000000,1931000000,...,9000000,2693000000,2852000000,339000000,11632000000,-2857000000,277000000,-2857000000,9391000000,8779000000


In [67]:
pd.DataFrame(ticker.all_modules['infy.ns']['balanceSheetHistoryQuarterly']['balanceSheetStatements'])

,maxAge,endDate,cash,shortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,longTermInvestments,propertyPlantEquipment,...,minorityInterest,totalCurrentLiabilities,totalLiab,commonStock,retainedEarnings,treasuryStock,capitalSurplus,otherStockholderEquity,totalStockholderEquity,netTangibleAssets
0,1,2022-12-31,1401000000,1055000000,5011000000,11000000,938000000,8753000000,1497000000,2444000000,...,51000000,4981000000,6200000000,327000000,12675000000,-4377000000,350000000,-4377000000,8975000000,7877000000
1,1,2022-09-30,1828000000,1448000000,4838000000,8000000,800000000,9211000000,1557000000,2379000000,...,49000000,4589000000,5763000000,328000000,13421000000,-4296000000,375000000,-4296000000,9828000000,8756000000
2,1,2022-06-30,1771000000,1027000000,4697000000,30000000,843000000,8682000000,1664000000,2419000000,...,50000000,4643000000,5806000000,328000000,12673000000,-4020000000,356000000,-4020000000,9337000000,8307000000
3,1,2022-03-31,2305000000,880000000,4609000000,25000000,783000000,8865000000,1801000000,2429000000,...,53000000,4433000000,5561000000,328000000,12842000000,-3566000000,337000000,-3566000000,9941000000,8899000000


In [79]:
ticker.summary_profile

{'matrimony.ns': {'address1': 'No. 94, TVH Beliciaa Towers',
  'address2': 'Tower 2, 5th Floor MRC Nagar Raja Annamalaipuram',
  'city': 'Chennai',
  'zip': '600028',
  'country': 'India',
  'phone': '91 44 4900 1919',
  'fax': '91 44 2463 1777',
  'website': 'https://www.matrimony.com',
  'industry': 'Internet Content & Information',
  'sector': 'Communication Services',
  'longBusinessSummary': 'Matrimony.com Limited, a consumer internet company, provides online matchmaking services on internet and mobile platforms in India and internationally. The company operates through two segments, Matchmaking Services and Marriage Services. It operates matrimonial sites, including BharatMatrimony.com, BengaliMatrimony.com, BihariMatrimony.com, GujaratiMatrimony.com, HindiMatrimony.com, KannadaMatrimony.com, KeralaMatrimony.com, MarathiMatrimony.com, MarwadiMatrimony.com, OriyaMatrimony.com, PunjabiMatrimony.com, RajasthaniMatrinomy.com, SindhiMatrimony.com, TamilMatrimony.com, TeluguMatrimony.c

In [71]:
ticker.financial_data

{'infy.ns': {'maxAge': 86400,
  'currentPrice': 1496.05,
  'targetHighPrice': 2402.07,
  'targetLowPrice': 1161.0,
  'targetMeanPrice': 1740.98,
  'targetMedianPrice': 1730.5,
  'recommendationMean': 1.9,
  'recommendationKey': 'buy',
  'numberOfAnalystOpinions': 44,
  'totalCash': 2456000000,
  'totalCashPerShare': 0.589,
  'ebitda': 4161999872,
  'totalDebt': 933000000,
  'quickRatio': 1.492,
  'currentRatio': 1.757,
  'totalRevenue': 17937000448,
  'debtToEquity': 10.337,
  'revenuePerShare': 4.278,
  'returnOnAssets': 0.15837,
  'returnOnEquity': 0.32488,
  'grossProfits': 5205000000,
  'freeCashflow': 2524499968,
  'operatingCashflow': 2889999872,
  'earningsGrowth': 0.056,
  'revenueGrowth': 0.096,
  'grossMargins': 0.29693002,
  'ebitdaMargins': 0.23202999,
  'operatingMargins': 0.21118,
  'profitMargins': 0.16664,
  'financialCurrency': 'USD'}}